In [1]:
!git clone https://github.com/chunxiangzheng/gaussian_log_file_converter

Cloning into 'gaussian_log_file_converter'...
remote: Enumerating objects: 19, done.
remote: Total 19 (delta 0), reused 0 (delta 0), pack-reused 19
Receiving objects: 100% (19/19), 2.47 MiB | 4.88 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [26]:
!python gaussian_log_file_converter/extractOptimizedCoords.py test.log gjf


In [27]:
!python gaussian_log_file_converter/extractOptimizedCoords.py test.log xyz

In [1]:
import os

def convert_gjf_format(input_filename, output_folder, mem='16GB', nprocshared='16', method='PBE1PBE', basis_set='def2TZVP'):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Construct the output filename
    output_filename = os.path.join(output_folder, os.path.basename(input_filename))
    
    with open(input_filename, 'r') as gjf_in_file:
        gjf_content = gjf_in_file.readlines()
    
    with open(output_filename, 'w') as gjf_out_file:
        # Write %mem and %nprocshared lines
        gjf_out_file.write(f'%mem={mem}\n')
        gjf_out_file.write(f'%nprocshared={nprocshared}\n')
        
        # Write Gaussian input line
        gaussian_input_line = f'# {method}/{basis_set} int=(grid=ultrafine) empiricaldispersion=GD3BJ pop=nbo7 prop=efg nmr=giao density=current output=wfn\n'
        gjf_out_file.write(gaussian_input_line)
        
        # Write remaining content from original .gjf file
        gjf_out_file.writelines(gjf_content[4:])  # Skip the first three lines which include %chk and blank lines

In [2]:
# Example usage:
input_filename = 'test_optimized_out.gjf'  # Replace with your actual input .gjf file
output_folder = '2ndRound_gjf'  # Replace with your desired output folder
mem = '16GB'  # Desired memory specification
nprocshared = '16'  # Desired processor specification
method = 'PBE1PBE'  # Desired Gaussian method
basis_set = 'def2TZVP'  # Desired basis set

convert_gjf_format(input_filename, output_folder, mem, nprocshared, method, basis_set)

In [36]:
import os
import re
import glob

def getEnergy(structure):
    for line in structure.split("\n"):
        if line.startswith(" SCF Done:"):
            arr = line.split("=")
            return float(re.split(" +", arr[1].strip())[0])
    return 1000.0

def findInList(dataList, target):
    for i in range(0, len(dataList)):
        if dataList[i].find(target) != -1:
            return i
    return -1

def getCoordinates(dataList):
    start = findInList(dataList, "Standard orientation")
    dataList = dataList[start + 5:]
    dataList = dataList[:findInList(dataList, "-----")]
    return dataList

def getChargeAndMultiplicity(infoBlock):
    lines = infoBlock.split("\n")
    for line in lines:
        if line.startswith(" Charge = "):
            arr = re.split(" +", line.strip())
            return (int(arr[2]), int(arr[5]))
    return (-1, -1)

code = {"1": "H", "2": "He", "3": "Li", "4": "Be", "5": "B",
        "6": "C", "7": "N", "8": "O", "9": "F", "10": "Ne",
        "11": "Na", "12": "Mg", "13": "Al", "14": "Si", "15": "P",
        "16": "S", "17": "Cl", "18": "Ar", "19": "K", "20": "Ca",
        "21": "Sc", "22": "Ti", "23": "V", "24": "Cr", "25": "Mn",
        "26": "Fe", "27": "Co", "28": "Ni", "29": "Cu", "30": "Zn",
        "31": "Ga", "32": "Ge", "33": "As", "34": "Se", "35": "Br",
        "36": "Kr", "37": "Rb", "38": "Sr", "39": "Y", "40": "Zr",
        "41": "Nb", "42": "Mo", "43": "Tc", "44": "Ru", "45": "Rh",
        "46": "Pd", "47": "Ag", "48": "Cd", "49": "In", "50": "Sn",
        "51": "Sb", "52": "Te", "53": "I", "54": "Xe", "55": "Cs",
        "56": "Ba", "57": "La", "58": "Ce", "59": "Pr", "60": "Nd",
        "61": "Pm", "62": "Sm", "63": "Eu", "64": "Gd", "65": "Tb",
        "66": "Dy", "67": "Ho", "68": "Er", "69": "Tm", "70": "Yb",
        "71": "Lu", "72": "Hf", "73": "Ta", "74": "W", "75": "Re",
        "76": "Os", "77": "Ir", "78": "Pt", "79": "Au", "80": "Hg",
        "81": "Tl", "82": "Pb", "83": "Bi", "84": "Po", "85": "At",
        "86": "Rn", "87": "Fr", "88": "Ra", "89": "Ac", "90": "Th",
        "91": "Pa", "92": "U", "93": "Np", "94": "Pu", "95": "Am",
        "96": "Cm", "97": "Bk", "98": "Cf", "99": "Es", "100": "Fm",
        "101": "Md", "102": "No", "103": "Lr", "104": "Rf", "105": "Db",
        "106": "Sg", "107": "Bh", "108": "Hs", "109": "Mt", "110": "Ds",
        "111": "Rg", "112": "Uub", "113": "Uut", "114": "Uuq", "115": "Uup",
        "116": "Uuh", "117": "Uus", "118": "Uuo"}

def convert_log_files(input_folder):
    output_folder = os.path.join(input_folder, 'converted')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    log_files = glob.glob(os.path.join(input_folder, '*.log'))

    for log_file in log_files:
        with open(log_file, "r") as fin:
            infoBlock = ""
            optimized = False
            optimized_structure = ""
            isStructure = True
            isInfo = True
            structures = []
            currentStructure = ""
            for line in fin:
                if line.startswith(" GradGrad"):
                    if isInfo:
                        isInfo = False
                    if currentStructure != "" and currentStructure.startswith(" GradGrad"):
                        structures.append((getEnergy(currentStructure), currentStructure))
                        currentStructure = ""
                    isStructure = not isStructure
                elif isInfo:
                    infoBlock += line
                elif isStructure:
                    currentStructure += line
                else:
                    if line.find("Optimized") != -1:
                        optimized = True

            if optimized:
                optimized_structure = currentStructure
            else:
                if currentStructure != "" and currentStructure.startswith(" GradGrad"):
                    structures.append((getEnergy(currentStructure), currentStructure))
                structures = sorted(structures, key=lambda item: item[0])
                optimized_structure = structures[0][1]

        prefix = os.path.splitext(os.path.basename(log_file))[0]
        foutput_xyz = os.path.join(output_folder, f"{prefix}_optimized_out.xyz")
        foutput_gjf = os.path.join(output_folder, f"{prefix}_optimized_out.gjf")

        dataList = optimized_structure.split("\n")
        atoms = getCoordinates(dataList)

        with open(foutput_xyz, "w") as fout:
            fout.write(str(len(atoms)) + "\n\n")
            for atom in atoms:
                arr = atom.split()
                symbol = code.get(arr[1], 'X')
                fout.write("  %s %16.7f %16.7f %16.7f\n" % (symbol, float(arr[3]), float(arr[4]), float(arr[5])))

        with open(foutput_gjf, "w") as fout:
            chk = f"{prefix}_optimized_out.chk"
            fout.write("%mem=\n%nprocshared=\n%chk=" + chk + "\n# \n\nComplex " + prefix + "\n\n")
            charge, multiplicity = getChargeAndMultiplicity(infoBlock)
            fout.write(str(charge) + " " + str(multiplicity) + "\n")
            for atom in atoms:
                arr = atom.split()
                symbol = code.get(arr[1], 'X')
                fout.write("  %s %16.7f %16.7f %16.7f\n" % (symbol, float(arr[3]), float(arr[4]), float(arr[5])))
            fout.write("\n")

# Example usage: specify your input folder
input_folder = 'ligands/l39_gjf'
convert_log_files(input_folder)


In [68]:
import os
import glob

def convert_gjf_format(input_filename, output_folder, mem='16GB', nprocshared='16', method='PBE1PBE', basis_set='def2TZVP'):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    with open(input_filename, 'r') as gjf_in_file:
        gjf_content = gjf_in_file.readlines()
    
    for suffix, prop in zip(['nbo', 'efg', 'giao'], 
                            ['pop=nbo', 'prop=efg', 'nmr=giao']):
        # Construct the output filename
        output_filename = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(input_filename))[0]}_{suffix}.gjf")
        
        with open(output_filename, 'w') as gjf_out_file:
            # Write %mem and %nprocshared lines
            gjf_out_file.write(f'%mem={mem}\n')
            gjf_out_file.write(f'%nprocshared={nprocshared}\n')
            
            # Write Gaussian input line
            gaussian_input_line = f'# {method}/{basis_set} int=(grid=ultrafine) empiricaldispersion=GD3BJ {prop}\n'
            gjf_out_file.write(gaussian_input_line)
            
            # Write remaining content from original .gjf file
            gjf_out_file.writelines(gjf_content[4:])  # Skip the first three lines which include %chk and blank lines

def process_folder(input_folder, mem='16GB', nprocshared='16', method='PBE1PBE', basis_set='def2TZVP'):
    gjf_files = glob.glob(os.path.join(input_folder, '*.gjf'))
    
    for gjf_file in gjf_files:
        # Create a new folder named after the original .gjf file with 'gjffolder' added
        base_name = os.path.splitext(os.path.basename(gjf_file))[0]
        output_folder = os.path.join(input_folder, f"{base_name}_gjffolder")
        
        convert_gjf_format(gjf_file, output_folder, mem, nprocshared, method, basis_set)

# Example usage:
input_folder = 'ligands/l39_gjf/converted'  # Replace with your actual input folder containing .gjf files
mem = '16GB'  # Desired memory specification
nprocshared = '16'  # Desired processor specification
method = 'PBE1PBE'  # Desired Gaussian method
basis_set = 'def2TZVP'  # Desired basis set

process_folder(input_folder, mem, nprocshared, method, basis_set)
